In [2]:
# initial setup
try:
    # settings colab:
    import google.colab
    
    # si usan colab, deben cambiar el token de esta url
    #! mkdir -p ../data
    # los que usan colab deben modificar el token de esta url:
    #! wget -O ../data/properati_2017.csv https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_students_2020/master/M2/CLASE_07_Pandas_2/Data/resultado-de-encuestas-2017-2018.csv?token=AA4GFHO3EKGWGILBCV4BISC6WR6B4
    
except ModuleNotFoundError:    
    # settings local:
    %run "common/base_setup.py"

Running command `conda list`... ok
pandas=1.0.3 already installed
matplotlib=2.2.2 already installed
bokeh=2.0.0 already installed
seaborn=0.10.0 already installed
ipywidgets=7.5.1 already installed
pytest=5.3.4 already installed
chardet=3.0.4 already installed
plotly=4.6.0 already installed
Running command `conda install --yes chart-studio=1.1.0`... error
    
PackagesNotFoundError: The following packages are not available from current channels:

  - chart-studio=1.1.0

Current channels:

  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.





## Estructura del Dataset

In [506]:
import pandas as pd
import numpy as np
import re

In [629]:
# low_memorybool, default True
# Internally process the file in chunks, resulting in lower memory use while parsing, 
# but possibly mixed type inference. To ensure no mixed types either set False, 
# or specify the type with the dtype parameter. 
# Note that the entire file is read into a single DataFrame regardless, 
# use the chunksize or iterator parameter to return the data in chunks. (Only valid with C parser).
data = pd.read_csv("./data/properati_2017.csv", sep = ",", low_memory=False)

In [630]:
data.shape

(121220, 26)

In [631]:
data.columns

Index(['Unnamed: 0', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail'],
      dtype='object')

In [632]:
data.dtypes

Unnamed: 0                      int64
operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: objec

### Columnas Irrelevantes para el estudio - Fase 1

In [633]:
def drop_columns(df, columns):
    all_columns = set(df.columns)
    relevant_columns = all_columns.difference(set(columns))
    return df.loc[:, list(relevant_columns)]

data = drop_columns(data, ['Unnamed: 0', 'image_thumbnail', 'properati_url'])

### Visualización Preliminar

In [634]:
data.head(10)

,rooms,price,lat,state_name,property_type,country_name,floor,price_usd_per_m2,price_aprox_local_currency,description,...,lon,operation,price_aprox_usd,currency,title,expenses,lat-lon,place_name,price_per_m2,place_with_parent_names
0,NaN,62000.0,-34.661824,Capital Federal,PH,Argentina,NaN,1127.272727,1093959.00,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",...,-58.508839,sell,62000.00,USD,2 AMB TIPO CASA SIN EXPENSAS EN PB,NaN,"-34.6618237,-58.5088387",Mataderos,1550.000000,|Argentina|Capital Federal|Mataderos|
1,NaN,150000.0,-34.903883,Bs.As. G.B.A. Zona Sur,apartment,Argentina,NaN,NaN,2646675.00,Venta de departamento en décimo piso al frente...,...,-57.964330,sell,150000.00,USD,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,NaN,"-34.9038831,-57.9643295",La Plata,NaN,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|
2,NaN,72000.0,-34.652262,Capital Federal,apartment,Argentina,NaN,1309.090909,1270404.00,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,...,-58.522982,sell,72000.00,USD,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,NaN,"-34.6522615,-58.5229825",Mataderos,1309.090909,|Argentina|Capital Federal|Mataderos|
3,NaN,95000.0,-34.647797,Capital Federal,PH,Argentina,NaN,NaN,1676227.50,PH 3 ambientes con patio. Hay 3 deptos en lote...,...,-58.516424,sell,95000.00,USD,PH 3 amb. cfte. reciclado,NaN,"-34.6477969,-58.5164244",Liniers,NaN,|Argentina|Capital Federal|Liniers|
4,NaN,64000.0,-38.002626,Buenos Aires Costa Atlántica,apartment,Argentina,NaN,1828.571429,1129248.00,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,...,-57.549447,sell,64000.00,USD,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,NaN,"-38.0026256,-57.5494468",Centro,1828.571429,|Argentina|Buenos Aires Costa Atlántica|Mar de...
5,NaN,NaN,-33.014071,Entre Ríos,house,Argentina,NaN,NaN,NaN,"Casa en el perímetro del barrio 338, ubicada e...",...,-58.519828,sell,NaN,NaN,"Casa Barrio 338. Sobre calle 3 de caballería, ...",NaN,"-33.0140714,-58.519828",Gualeguaychú,NaN,|Argentina|Entre Ríos|Gualeguaychú|
6,NaN,130000.0,-34.532957,Bs.As. G.B.A. Zona Norte,PH,Argentina,NaN,1226.415094,2293785.00,MUY BUEN PH AL FRENTE CON ENTRADA INDEPENDIENT...,...,-58.521782,sell,130000.00,USD,"MUY BUEN PH AL FRENTE DOS DORMITORIOS , PATIO,...",NaN,"-34.5329567,-58.5217825",Munro,1666.666667,|Argentina|Bs.As. G.B.A. Zona Norte|Vicente Ló...
7,NaN,138000.0,-34.559873,Capital Federal,apartment,Argentina,NaN,3066.666667,2434941.00,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,...,-58.443362,sell,138000.00,USD,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,NaN,"-34.5598729,-58.443362",Belgrano,3450.000000,|Argentina|Capital Federal|Belgrano|
8,NaN,195000.0,-34.559873,Capital Federal,apartment,Argentina,NaN,3000.000000,3440677.50,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,...,-58.443362,sell,195000.00,USD,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",NaN,"-34.5598729,-58.443362",Belgrano,3250.000000,|Argentina|Capital Federal|Belgrano|
9,NaN,460000.0,-32.942031,Santa Fe,house,Argentina,NaN,NaN,455201.45,MEDNOZA AL 7600A UNA CUADRA DE CALLE MENDOZAWH...,...,-60.725919,sell,25798.49,ARS,WHITE 7637 - 2 DORMITORIOS CON PATIO,NaN,"-32.942031,-60.7259192",Rosario,NaN,|Argentina|Santa Fe|Rosario|


### Distribución de N/A

In [635]:
def show_null_distribution(df):
    nulls = df.isnull().sum(axis=0)
    nulls_percent = nulls / df.index.size
    nulls_df = pd.concat([nulls, nulls_percent], axis=1)
    nulls_df.columns = ['total', 'percent']
    return nulls_df[nulls_df['total'] > 0] # solo me interesan las columnas que tengan N/A

show_null_distribution(data)

,total,percent
rooms,73830,0.609058
price,20410,0.168372
lat,51550,0.425260
floor,113321,0.934837
price_usd_per_m2,52603,0.433947
price_aprox_local_currency,20410,0.168372
description,2,0.000016
surface_total_in_m2,39328,0.324435
geonames_id,18717,0.154405
surface_covered_in_m2,19907,0.164222


## Tratamiento de Valores Faltantes

### Columna *place_name*

Observamos que la columna *place_with_parent_names* se construye concatenando *country_name*, *state_name* y *place_name*, entonces chequeamos esa columna contra los faltantes:

In [636]:
na_place_name_mask = data["place_name"].isnull()
na_place_name      = data[na_place_name_mask][["place_name", "place_with_parent_names"]]
na_place_name.drop_duplicates()

,place_name,place_with_parent_names
6489,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||


Vemos que en este caso los N/A son de **Tigre** 

In [637]:
data['place_name'] = data['place_name'].fillna("Tigre")

### Columnas *description* y *title*

In [638]:
na_description_mask = data["description"].isnull()
na_description      = data[na_description_mask]
na_description

,rooms,price,lat,state_name,property_type,country_name,floor,price_usd_per_m2,price_aprox_local_currency,description,...,lon,operation,price_aprox_usd,currency,title,expenses,lat-lon,place_name,price_per_m2,place_with_parent_names
10664,NaN,2350000.0,-31.420783,Córdoba,apartment,Argentina,NaN,NaN,2325485.81,NaN,...,-64.499214,sell,131796.64,ARS,HERMOSO DEPARTAMENTO CON PISCINA Y COCHERA,NaN,"-31.4207828,-64.4992141",Villa Carlos Paz,NaN,|Argentina|Córdoba|Villa Carlos Paz|
96691,NaN,98000.0,NaN,Bs.As. G.B.A. Zona Norte,apartment,Argentina,NaN,NaN,1729161.00,NaN,...,NaN,sell,98000.00,USD,Departamento - San Fernando,NaN,NaN,San Fernando,1781.818182,|Argentina|Bs.As. G.B.A. Zona Norte|San Fernando|


Por ahora, solo las normalizamos:

In [639]:
rex = re.compile(r'\s+')
fformat = lambda x: rex.sub(' ', str(x)).strip().lower()
data['description'] = data['description'].apply(fformat)
data['title']       = data['title'].apply(fformat)

### Columna *expenses*
En este caso, chequeamos la cantidad de N/A contra la cantidad de posibles calculables.
El criterio de la esa posibilidad, es simple: la palabra **expensa** dentro de *title* o *description*.

In [640]:
na_expenses_mask = data['expenses'].isnull()
na_expenses = data[na_expenses_mask]
na_expenses_total = na_expenses_mask.sum()
calc_expenses_total = na_expenses['title'].str.contains('expensa', case=False, regex=False).sum() + \
   na_expenses['description'].str.contains('expensa', case=False, regex=False).sum() 

print("Proportion of missing expenses: {0}".format(na_expenses_total / data.index.size))
print("Proportion of posibles calculable expenses: {0}".format(calc_expenses_total / na_expenses_total))

Proportion of missing expenses: 0.8823461475004125
Proportion of posibles calculable expenses: 0.12412348772415341


De lo anterior, concluimos que casi un 90% de los datos son N/A y de ese porcentage menos de un 13% tiene alguna posibilidad de ser inferido. Por ende, no la vamos a tener en cuenta para hacer el estudio.

### Columna *floor*
Al igual que en el caso anterior, lo primero es chequear la proporción de N/A.

In [641]:
na_floor_mask = data['floor'].isnull()
na_floor = data[na_floor_mask]
na_floor_total = na_floor.index.size
print("Proportion of missing floors: {0}".format(na_floor_total / data.index.size))

Proportion of missing floors: 0.9348374855634384


Es una proporción alta aunque creemos que este campo tiene más relevancia cuando el *property_type* es **apartment**.

In [642]:
data_apartment_df = data[data['property_type'] == 'apartment']
na_apartment_floor_mask = data_apartment_df['floor'].isnull()
na_apartment_floor_total = na_apartment_floor_mask.sum()
print("Total of apartment properties: {0}".format(data_apartment_df.index.size))
print("Proportion of missing floors: {0}".format(na_apartment_floor_total / data_apartment_df.index.size))

Total of apartment properties: 71065
Proportion of missing floors: 0.9082389361851826


Nuevamente, la proporción de N/A es muy alta. Por ende, esta columna también la vamos a descartar del estudio.

### Columna *rooms*
De nuevo, chequeamos la proporción de N/A.

In [643]:
na_rooms_mask = data['rooms'].isnull()
na_rooms = data[na_rooms_mask]
na_rooms_total = na_rooms.index.size
print("Proportion of missing rooms: {0}".format(na_rooms_total / data.index.size))

Proportion of missing rooms: 0.6090579112357697


Otra vez, nos encontramos con una proporción alta (aunque no tal alta como los casos anteriores).
En este caso, la descripción en muchos casos tiene información que podría ser usada para inferir la cantidad de ambientes que tiene una propiedad pero habría que programar lógica capaz de extraer dicha información, sumarizarla y en muchos casos chequearla personalmente.

Desestimamos incluir esta columna en este primer análisis debido a la carga de trabajo que supone lo explicado en el párrafo anterior.

### Columna *price* y relacionadas
Vamos a dividir el dataset en 2 partes:
 * el precio está disponible o puede inferirse a partir de otras columnas.
 * el precio es N/A y las columnas de las que se podría inferir también son N/A.

In [644]:
def show_valid_rows_proportion(df):
    na_surface_mask     = data[['surface_total_in_m2', 'surface_covered_in_m2']].isna().all(axis=1)
    na_total_price_mask = data[['price', 'price_aprox_local_currency', 'price_aprox_usd']].isna().all(axis=1)
    na_m2_price_mask    = data[['price_per_m2', 'price_usd_per_m2']].isna().all(axis=1)

    uncalculable_mask = (na_surface_mask & na_total_price_mask) | (na_surface_mask & na_m2_price_mask) | \
    (na_total_price_mask & na_m2_price_mask)
    
    df_calculables   = df[~uncalculable_mask]
    df_uncalculables = df[uncalculable_mask]
    print("Prices/Surfaces non calculables: {0}".format(df_uncalculables.index.size))
    print("Prices/Surfaces calculables: {0}".format(df_calculables.index.size))

show_valid_rows_proportion(data)

Prices/Surfaces non calculables: 28046
Prices/Surfaces calculables: 93174


Lo anterior significa que tenemos casi 30000 registros que carecen de los datos necesarios para ser tenidos en cuenta. Vamos a tratar de inferir algunos usando las columnas **title** y **description**.

In [645]:
def apply_price_title_and_description(r):
    def getprice(source, pattern):
        price = re.sub("[^0-9]", "", source[source.find(pattern) + len(pattern):].strip().split(' ')[0])
        return np.float64(price)
    if re.search(r'en\s+venta', r.title, re.I):
        if r.title.startswith(r'u$d') or r.title.startswith(r'usd'):
            r.price    = np.float64(r.title.split()[1].strip().replace('.', ''))
            r.currency = 'USD'
        elif r.title.startswith(r'$'):
            r.price    = np.float64(r.title.split()[1].strip().replace('.', ''))
            r.currency = 'ARS'
    elif r'vendo $' in r.description:
        r.price    = getprice(r.description, 'vendo $')
        r.currency = 'ARS'
    elif r'vendo casa $' in r.description:
        r.price    = getprice(r.description, r'vendo casa $')
        r.currency = 'ARS'
    elif r'vendo u$s' in r.description:
        r.price    = getprice(r.description, 'vendo u$s')
        r.currency = 'USD'
    return r

price_na_mask = data['price'].isna()
data[price_na_mask] = data[price_na_mask].apply(apply_price_title_and_description, axis=1)

show_valid_rows_proportion(data)

Prices/Surfaces non calculables: 25480
Prices/Surfaces calculables: 95740


### Columnas Irrelevantes para el estudio - Fase 2
Procedemos a quitar del dataset las columnas que *expenses*, *floor* y *rooms*.
Quitamos además las columnas *operation* y *country_name* (tienen un solo valor) y *place_with_parent_names* dado que es la concatenación de otras 3 columnas.

In [646]:
data = drop_columns(data, ['country_name', 'operation', 'place_with_parent_names', 'expenses', 'floor', 'rooms'])
data.head(5)

,lat,surface_total_in_m2,geonames_id,surface_covered_in_m2,lon,price_aprox_usd,currency,state_name,price_aprox_local_currency,title,property_type,price_usd_per_m2,price,lat-lon,place_name,price_per_m2,description
0,-34.661824,55.0,3430787.0,40.0,-58.508839,62000.0,USD,Capital Federal,1093959.0,2 amb tipo casa sin expensas en pb,PH,1127.272727,62000.0,"-34.6618237,-58.5088387",Mataderos,1550.000000,"2 ambientes tipo casa planta baja por pasillo,..."
1,-34.903883,NaN,3432039.0,NaN,-57.964330,150000.0,USD,Bs.As. G.B.A. Zona Sur,2646675.0,venta depto 2 dorm. a estrenar 7 e/ 36 y 37,apartment,NaN,150000.0,"-34.9038831,-57.9643295",La Plata,NaN,venta de departamento en décimo piso al frente...
2,-34.652262,55.0,3430787.0,55.0,-58.522982,72000.0,USD,Capital Federal,1270404.0,2 amb 3er piso con ascensor apto credito,apartment,1309.090909,72000.0,"-34.6522615,-58.5229825",Mataderos,1309.090909,2 ambientes 3er piso lateral living comedor am...
3,-34.647797,NaN,3431333.0,NaN,-58.516424,95000.0,USD,Capital Federal,1676227.5,ph 3 amb. cfte. reciclado,PH,NaN,95000.0,"-34.6477969,-58.5164244",Liniers,NaN,ph 3 ambientes con patio. hay 3 deptos en lote...
4,-38.002626,35.0,3435548.0,35.0,-57.549447,64000.0,USD,Buenos Aires Costa Atlántica,1129248.0,depto 2 amb al contrafrente zona centro/plaza ...,apartment,1828.571429,64000.0,"-38.0026256,-57.5494468",Centro,1828.571429,departamento con fantástica iluminación natura...


## Tratamiento de duplicados
El primer lote de duplicados que consideramos se compone por el siguiente conjunto de columnas:
 * property_type
 * lat
 * lon
 * price
 * place_name
 * geonames_id

Comenzamos identificando duplicados sin tener en cuenta las filas con N/A:

In [647]:
with_duplicates_ds    = data[data[['lat', 'lon', 'geonames_id', 'price']].notna().all(axis=1)]
without_duplicates_ds = with_duplicates_ds.drop_duplicates(subset=('property_type', 'lat', 'lon', 'price', 'place_name', 'geonames_id'), keep='first')
print("Total with duplicates: {0}".format(with_duplicates_ds.index.size))
print("Total without duplicates: {0}".format(without_duplicates_ds.index.size))

Total with duplicates: 51041
Total without duplicates: 46949


El siguiente paso es eliminar efectivamente los duplicados de la celda anterior.

In [648]:
duplicates_mask = data[['property_type', 'lat', 'lon', 'price', 'place_name', 'geonames_id']].duplicated()
not_na_mask     = data[['lat', 'lon', 'geonames_id', 'price']].notna().all(axis=1)
data = data[~(duplicates_mask & not_na_mask)]

data.shape

(117128, 17)

In [649]:
show_null_distribution(data)

,total,percent
lat,51550,0.440117
surface_total_in_m2,38404,0.327881
geonames_id,18717,0.159800
surface_covered_in_m2,19313,0.164888
lon,51550,0.440117
price_aprox_usd,20275,0.173101
currency,17787,0.151860
price_aprox_local_currency,20275,0.173101
price_usd_per_m2,51536,0.439997
price,17786,0.151851


### Tratamiento de Valores Faltantes para las columnas de coordenadas
Una vez utilizadas las columnas *lat* y *lon* para determinar el duplicado de las filas, chequeamos los valores faltantes.

In [650]:
lat_lon_mask = data[['lat', 'lon', 'lat-lon']].isna().all(axis=1)
print("Total of null coordinates: {0}".format(lat_lon_mask.sum()))

Total of null coordinates: 51550


Podríamos localizar algunos de los N/A escaneando las columna *description*:

In [651]:
calc_lat_total = na_expenses['description'].str.contains('lat', case=False, regex=False).sum()
calc_lon_total = na_expenses['description'].str.contains('lon', case=False, regex=False).sum()
calc_lng_total = na_expenses['description'].str.contains('lng', case=False, regex=False).sum()
calc_lat_total, calc_lon_total, calc_lng_total

(8263, 4352, 1)

Dado que no es común que la descripción incluya la dirección real del inmueble, en el mejor de los casos (siendo muy optimistas) nos quedarían más de 40000 registros con N/A.

### Columnas Irrelevantes para el estudio - Fase 3
Eliminamos las 3 columnas relacionadas a coordenadas:
* lat
* lon
* lat-lon

In [652]:
data = drop_columns(data, ['lat', 'lon', 'lat-lon'])
data.head(5)

,state_name,property_type,price_usd_per_m2,price_aprox_local_currency,description,surface_total_in_m2,geonames_id,surface_covered_in_m2,price_aprox_usd,currency,title,price,place_name,price_per_m2
0,Capital Federal,PH,1127.272727,1093959.0,"2 ambientes tipo casa planta baja por pasillo,...",55.0,3430787.0,40.0,62000.0,USD,2 amb tipo casa sin expensas en pb,62000.0,Mataderos,1550.000000
1,Bs.As. G.B.A. Zona Sur,apartment,NaN,2646675.0,venta de departamento en décimo piso al frente...,NaN,3432039.0,NaN,150000.0,USD,venta depto 2 dorm. a estrenar 7 e/ 36 y 37,150000.0,La Plata,NaN
2,Capital Federal,apartment,1309.090909,1270404.0,2 ambientes 3er piso lateral living comedor am...,55.0,3430787.0,55.0,72000.0,USD,2 amb 3er piso con ascensor apto credito,72000.0,Mataderos,1309.090909
3,Capital Federal,PH,NaN,1676227.5,ph 3 ambientes con patio. hay 3 deptos en lote...,NaN,3431333.0,NaN,95000.0,USD,ph 3 amb. cfte. reciclado,95000.0,Liniers,NaN
4,Buenos Aires Costa Atlántica,apartment,1828.571429,1129248.0,departamento con fantástica iluminación natura...,35.0,3435548.0,35.0,64000.0,USD,depto 2 amb al contrafrente zona centro/plaza ...,64000.0,Centro,1828.571429


## Tratamiento de N/A
Para comenzar, identificamos todos los registros que no tienen referencia alguna del precio:

In [653]:
na_price_mask = data[['price', 'price_aprox_local_currency', 'price_aprox_usd', 'price_per_m2', 'price_usd_per_m2']].isna().all(axis=1)
na_price_mask.sum()

17786

Identifiquemos tambien todos los registros que no tienen referencia alguna a los metros cuadrados:

In [654]:
na_surface_mask = data[['surface_total_in_m2', 'surface_covered_in_m2']].isna().all(axis=1)
na_surface_mask.sum()

12080

Y cuantos tienen carecen de ambas:

In [655]:
data[na_price_mask & na_surface_mask].index.size

4675

Eliminamos los que carecen de ambas dado que de inferir, habría que hacerlo con todos las columnas.

In [656]:
columns_price = ['price', 'price_aprox_local_currency', 'price_aprox_usd', 'price_per_m2', 'price_usd_per_m2', 'surface_total_in_m2', 'surface_covered_in_m2']
data.dropna(subset=columns_price, how='all', inplace=True)

Ahora verificamos que registros tienen la información necesaria para inferir o aproximar las columnas de interés. Se pueden dar los siguientes casos:
* Con una superficie y un precio total, podemos inferir el precio por m2 (en el peor de los casos usaríamos la superficie cubierta para aproximar la superficie total).
* Con un precio total y un precio por m2, podemos inferir la superficie (en el peor de los casos la superficie inferida sería considerada total)
* Con una superficie y un precio por m2, podemos inferir el precio total (en el peor de los casos usaríamos la superficie cubierta para aproximar la superficie total).

Como valor del dolar, podemos tomar el promedio que tuvo la divisa durante 2017.

In [657]:
na_surface_mask     = data[['surface_total_in_m2', 'surface_covered_in_m2']].isna().all(axis=1)
na_total_price_mask = data[['price', 'price_aprox_local_currency', 'price_aprox_usd']].isna().all(axis=1)
na_m2_price_mask    = data[['price_per_m2', 'price_usd_per_m2']].isna().all(axis=1)

drop_condition_mask = (na_surface_mask & na_total_price_mask) | (na_surface_mask & na_m2_price_mask) | \
(na_total_price_mask & na_m2_price_mask)

drop_condition_mask.sum()

20516

Eliminamos las filas que no tienen la información necesaria para proceder según los puntos anteriores.

In [658]:
data = data.loc[~drop_condition_mask, :]
data.shape

(91937, 14)

In [660]:
data.describe()

,price_usd_per_m2,price_aprox_local_currency,surface_total_in_m2,geonames_id,surface_covered_in_m2,price_aprox_usd,price,price_per_m2
count,65592.000000,8.950500e+04,67963.000000,7.521800e+04,86696.000000,8.950500e+04,9.193700e+04,8.429400e+04
mean,2139.819258,4.333787e+06,230.747421,3.549370e+06,131.226769,2.456169e+05,4.547133e+05,6.889991e+03
std,2720.115239,6.463777e+06,1730.492332,3.720210e+05,749.718499,3.663338e+05,8.898536e+05,2.866172e+04
min,0.600000,8.234017e+04,0.000000,3.427208e+06,0.000000,4.666620e+03,1.000000e+00,1.510204e+00
25%,1200.000000,1.623294e+06,50.000000,3.429979e+06,45.000000,9.200000e+04,1.100000e+05,1.533880e+03
50%,1787.469807,2.646675e+06,85.000000,3.433357e+06,75.000000,1.500000e+05,1.900000e+05,2.200000e+03
75%,2481.481481,4.869882e+06,206.000000,3.436109e+06,150.000000,2.760000e+05,4.100000e+05,3.355891e+03
max,206333.333333,8.212711e+08,200000.000000,6.948895e+06,187000.000000,4.654544e+07,1.000000e+08,4.000000e+06
